In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import spacy
from spacy.tokens import Doc
from spacy.tokens import DocBin

In [3]:
directory='/content/drive/MyDrive/DataSemantic_Project/3doc/'

In [4]:
trained_nlp = spacy.load("/content/drive/MyDrive/DataSemantic_Project/outpu2/model-best")

In [5]:
docbin=DocBin()
for filename in os.listdir(directory):
    with open(directory+filename) as f:
      a=''.join(f.readlines())
    a=a.replace('\n', ' ').replace('\t', ' ').lower()
    doc=trained_nlp(a)
    docbin.add(doc)
docbin.to_disk('/content/drive/MyDrive/DataSemantic_Project/provaBS.spacy')

In [18]:
directory1='/content/drive/MyDrive/DataSemantic_Project/pred_gold_doc/validation/'

In [19]:
docbin_v=DocBin()
for filename in os.listdir(directory1):
    with open(directory1+filename) as f:
      a=''.join(f.readlines())
    a=a.replace('\n', ' ').replace('\t', ' ').lower()
    doc=trained_nlp(a)
    docbin_v.add(doc)
docbin_v.to_disk('/content/drive/MyDrive/DataSemantic_Project/provaBS_val.spacy')

In [6]:
import srsly
import typer
import warnings
from pathlib import Path

import spacy
from spacy.tokens import DocBin

In [7]:
!python -m spacy init fill-config /content/drive/MyDrive/DataSemantic_Project/base_configBS.cfg /content/drive/MyDrive/DataSemantic_Project/configBS.cfg

2023-06-15 14:43:31.088164: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/DataSemantic_Project/configBS.cfg
You can now add your data and train your pipeline:
python -m spacy train configBS.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
!python -m spacy train /content/drive/MyDrive/DataSemantic_Project/configBS.cfg --output /content/drive/MyDrive/DataSemantic_Project/output3_testBS

2023-06-15 14:44:06.393689: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory:
/content/drive/MyDrive/DataSemantic_Project/output3_testBS
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-06-15 14:44:09,341] [INFO] Set up nlp object from config
[2023-06-15 14:44:09,364] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-06-15 14:44:09,369] [INFO] Created vocabulary
[2023-06-15 14:44:09,371] [INFO] Finished initializing nlp object
[2023-06-15 14:44:14,984] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00   2227.71    0.00    0.00    0.00    0.00
 66

In [9]:
import typer
from pathlib import Path

import spacy

In [10]:
def create_config(model_name: str, component_to_update: str, output_path: Path):
    nlp = spacy.load(model_name)

    # create a new config as a copy of the loaded pipeline's config
    config = nlp.config.copy()

    # revert most training settings to the current defaults
    default_config = spacy.blank(nlp.lang).config
    config["corpora"] = default_config["corpora"]
    config["training"]["logger"] = default_config["training"]["logger"]

    # copy tokenizer and vocab settings from the base model, which includes
    # lookups (lexeme_norm) and vectors, so they don't need to be copied or
    # initialized separately
    config["initialize"]["before_init"] = {
        "@callbacks": "spacy.copy_from_base_model.v1",
        "tokenizer": model_name,
        "vocab": model_name,
    }
    config["initialize"]["lookups"] = None
    config["initialize"]["vectors"] = None

    # source all components from the loaded pipeline and freeze all except the
    # component to update; replace the listener for the component that is
    # being updated so that it can be updated independently
    config["training"]["frozen_components"] = []
    for pipe_name in nlp.component_names:
        if pipe_name != component_to_update:
            config["components"][pipe_name] = {"source": model_name}
            config["training"]["frozen_components"].append(pipe_name)
        else:
            config["components"][pipe_name] = {
                "source": model_name,
                "replace_listeners": ["model.tok2vec"],
            }

    # save the config
    config.to_disk(output_path)


#if __name__ == "__main__":
 #   typer.run(create_config)

In [13]:
create_config("/content/drive/MyDrive/DataSemantic_Project/outpu2/model-best", 'ner', '/content/drive/MyDrive/DataSemantic_Project/prova_config.cfg')

In [20]:
!python -m spacy train /content/drive/MyDrive/DataSemantic_Project/prova_config.cfg --output /content/drive/MyDrive/DataSemantic_Project/output3_testBS

2023-06-15 16:01:14.601639: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory:
/content/drive/MyDrive/DataSemantic_Project/output3_testBS
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-06-15 16:01:18,185] [INFO] Set up nlp object from config
[2023-06-15 16:01:18,208] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-06-15 16:01:18,209] [INFO] Resuming training for: ['ner']
[2023-06-15 16:01:18,222] [INFO] Copying tokenizer from: /content/drive/MyDrive/DataSemantic_Project/outpu2/model-best
[2023-06-15 16:01:19,020] [INFO] Copying vocab from: /content/drive/MyDrive/DataSemantic_Project/outpu2/model-best
[2023-06-15 16:01:19,056] [INFO] Created vocabulary
[2023-06-15 16:01:19,058] [INFO] Finished initializing nlp object
[2023-06-15 16:01:19,058] [INFO] Initialized pipeline components: []
✔ Initialized pipeline

============================= Training pipeline ==============